# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770815816465                   -0.53    9.0         
  2   -2.772142804552       -2.88       -1.31    1.0    154ms
  3   -2.772170327912       -4.56       -2.54    1.0    182ms
  4   -2.772170699471       -6.43       -3.44    2.0    176ms
  5   -2.772170721020       -7.67       -3.90    1.0    170ms
  6   -2.772170722856       -8.74       -4.82    1.0    191ms
  7   -2.772170723008       -9.82       -5.05    2.0    185ms
  8   -2.772170723015      -11.16       -6.50    1.0    182ms
  9   -2.772170723015      -12.94       -6.86    2.0    201ms
 10   -2.772170723015      -14.05       -7.98    1.0    182ms
 11   -2.772170723015      -14.75       -8.82    2.0    202ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770548509747                   -0.53    8.0         
  2   -2.7

1.7735580394577344

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770726216029                   -0.53    8.0         
  2   -2.772039806771       -2.88       -1.30    1.0    222ms
  3   -2.772081346381       -4.38       -2.68    1.0    157ms
  4   -2.772083409566       -5.69       -3.67    2.0    232ms
  5   -2.772083416383       -8.17       -3.99    2.0    180ms
  6   -2.772083417777       -8.86       -5.26    1.0    163ms
  7   -2.772083417810      -10.48       -5.69    2.0    218ms
  8   -2.772083417811      -12.34       -6.37    2.0    191ms
  9   -2.772083417811      -13.68       -6.76    1.0    192ms
 10   -2.772083417811   +  -14.10       -7.77    1.0    181ms
 11   -2.772083417811      -14.12       -8.54    2.0    211ms

Polarizability via ForwardDiff:       1.772534973588074
Polarizability via finite difference: 1.7735580394577344
